# Pipple Lecture 9 -- Simpy
In deze Pipple lecture gaan we aan de slag met de basics van Simpy. Meer informatie over Simpy, waaronder handige voorbeelden die vandaag zeker van pas kunnen komen, kan je vinden op https://simpy.readthedocs.io/en/latest/.
Voor je Simpy kunt gebruiken moet je het met onderstaande code installeren in Colab, omdat Google Colab Simpy niet als standaard package heeft.




In [0]:
!pip install simpy # Google Colab kent simpy niet dus handmatig installeren

Hieronder staat het carwash voorbeeld uit de lecture nog eens uitgewerkt.





In [0]:
import random
import simpy

def car(env, name, carwash):
  with carwash.request() as req:
    # Ask for carwash space
    yield req 
    print("%s started washing at %s" %(name, env.now))
    
    # Washing car takes between 15 and 20 minutes
    yield env.timeout(random.randint(15, 20))
    print("%s finished washing and left the carwash at %s" %(name, env.now))

def set_up(env):
  # Create carwash
  carwash = simpy.Resource(env, capacity=4)
  
  # Create initial 3 cars
  for i in range(3):
    env.process(car(env, "Car " + str(i), carwash))
    print("Car %s arrived at the carwash at %s" %(i, env.now))
    
  # Create remaining 7 cars, all with 5 minutes delay  
  for i in range(3, 10):
    yield env.timeout(5)
    env.process(car(env, "Car " + str(i), carwash))
    print("Car %s arrived at the carwash at %s" %(i, env.now))


env = simpy.Environment()
env.process(set_up(env))
env.run(100) # simulation time is 100 minutes



---

# Kennismaking met processen en resources
De basics van Simpy draaien om processen die gebruik maken van resources. Om het voor vandaag allemaal net iets leuker te houden zullen wij vandaag niet werken met een groot magazijn, maar met Pippelaars die aanwezig zijn in de karaoke bar. De setting is dan ook als volgt:


*   Drie van onze Pippelaars (processen) zijn bij aanvang van de simulatie al aanwezig in de karaoke bar
*   Gedurende de avond stromen er nog 4 Pippelaars binnen (hier zit een normaal verdeelde tijd tussen met een mean van 15 minuten en een standaardafwijking van 3 minuten).
*   De Pippelaars willen natuurlijk allemaal een liedje zingen en vragen hiervoor een plekje aan op het podium (resource), ongeveer 10 minuten nadat ze aangekomen zijn (met een standaardafwijking van 2 minuten).
*   Een liedje zingen kost 10 minuten. (Ja het zijn lange nummertjes deze avond)

**De opdracht:** Momenteel doen de Pippelaars niks anders dan 30 minuten chillen in de karaokebar, om vervolgens weer weg te gaan. (*Zie: def pippelaar(env, name, podium, color)*). Zorg dat de pippelaars ook daadwerkelijk een plekje op het podium aanvragen en print hierbij als output voor iedere pippelaar (in hun eigen kleur) op welk moment:


1.   Hij/zij een plekje op het podium aanvraagt
2.   Hij/zij ook daadwerkelijk begint met zingen
3.   Hij/zij klaar is met zingen



Onderstaand stukje code kan gebruikt worden als basis voor de simulatie. Om het overzicht niet kwijt te raken is het handig om deze cell te kopieren en in de gekopieerde versie dingen te proberen, op die manier blijft 'de basiscode' altijd beschikbaar.

In [0]:
import random
import simpy
from termcolor import colored


random.seed(42)
pippelaars = ['Ruud', 'Bart', 'Wouter', 'Roos', 'Erlijn', 'Gijs', 'Lars']
sim_time = 500  # Simulate until
colors = ['grey', 'red', 'green', 'blue', 'yellow', 'magenta', 'cyan'] # Kleuren worden toegevoegd zodat output in kleur geprint kan worden voor de overzichtelijkheid


def pippelaar(env, name, podium, color):
  """A pippelaar arrives at the karaokebar at a certain time and will request a
  podium space several minutes after his arrival """
  
  yield env.timeout(30)
  # Het enige wat de Pippelaar nu doet is 30 minuten chillen in de karaokebar, dan gaat hij weer weg
  
  print(colored("%s leaves the bar at %s" % (name, env.now), color))
    

def pippelaars_arrive(env, pippelaars, colors):
  """ Makes sure Pippelaars arrive at the karaoke bar"""
  podium = simpy.Resource(env, capacity=1) # Resource wordt aangemaakt
  
  for i in range(3):
    # Drie eerste Pippelaars arriveren in de karaokebar
    env.process(pippelaar(env, pippelaars[i], podium, colors[i]))
    print(colored("%s arrived at the karaokebar at %s" % (pippelaars[i], env.now), colors[i]))
    
  for i in range(3, 7):
    yield env.timeout(random.normalvariate(15, 3))
    env.process(pippelaar(env, pippelaars[i], podium, colors[i]))
    print(colored("%s arrived at the karaokebar at %s" % (pippelaars[i], env.now), colors[i]))
    
env = simpy.Environment()
env.process(pippelaars_arrive(env, pippelaars, colors))
env.run(until=sim_time)


# Priority resources
Terwijl de Pippelaars met zijn allen een gezellige tijd hebben in de karaokebar besluit ook ons oprichtertje zijn gezicht te laten zien. Jeroen is echter al zo vaak in de karaokebar geweest dat hij geleerd heeft dat je beter direct bij de DJ een nummer aan kan vragen dan je nummer op te schrijven op een papiertje bij de bar. De nummers die Jeroen aanvraagt komen dan ook altijd boven aan de lijst te staan bij de DJ en hij is altijd eerder aan de beurt dan de andere Pippelaars.

**De opdracht:** Zorg dat ook ons oprichtertje arriveert in de karaokebar, en zorg ervoor dat de podium resource voorrang kan geven aan verzoekjes die Jeroen doet. Zorg dat Jeroen ergens tussen de 15 en 45 minuten aankomt in de karaokebar, ook hij wacht 10 minuten (met een afwijking van 2 minuten) tot hij een nummer aan gaat vragen. Maar als hij dan eenmaal een nummer gaat aanvragen komt hij vooraan in de wachtrij. Een opzet voor hoe Jeroen in het set_up process verwerkt kan worden is gedaan in de volgende cell.

In [0]:
import random
import simpy
from termcolor import colored


random.seed(42)
pippelaars = ['Ruud', 'Bart', 'Wouter', 'Roos', 'Erlijn', 'Gijs', 'Lars']
sim_time = 500  # Simulate until
colors = ['grey', 'red', 'green', 'blue', 'yellow', 'magenta', 'cyan'] # Kleuren worden toegevoegd zodat output in kleur geprint kan worden voor de overzichtelijkheid

def pippelaar(env, name, podium, color):
  """A pippelaar arrives at the karaokebar at a certain time and will request a
  podium space several minutes after his arrival """
  
  yield env.timeout(30)
  # Het enige wat de Pippelaar nu doet is 30 minuten chillen in de karaokebar, dan gaat hij weer weg
  # --> Gebruik hier je code van de vorige opdracht
  
  print(colored("%s leaves the bar at %s" % (name, env.now), color))

def set_up(env, pippelaars, colors):
  """ Makes sure Pippelaars arrive at the karaoke bar"""
  podium = simpy.Resource(env, capacity=1)
  env.process(jeroen_arriveert(env, podium))
  
  for i in range(3):
    env.process(pippelaar(env, pippelaars[i], podium, colors[i]))
    print(colored("%s arrived at the karaokebar at %s" % (pippelaars[i], env.now), colors[i]))
    
  for i in range(3, 7):
    yield env.timeout(random.normalvariate(15, 3))
    env.process(pippelaar(env, pippelaars[i], podium, colors[i]))
    print(colored("%s arrived at the karaokebar at %s" % (pippelaars[i], env.now), colors[i]))

def jeroen_arriveert(env, podium):
  """Het oprichtertje verdient een apart simpy process om zijn gedrag te simuleren, omdat het anders is dan dat van de andere pippelaars"""
  yield env.timeout(1600) # Nu blijft Jeroen eigenlijk heel de avond door het raam naar binnen kijken zonder de bar in te komen
  # Simuleer zijn aankomst tijd

  # Ook het oprichtertje heeft even tijd nodig om binnen te komen en zal pas na ongeveer 10 minuten een nummertje aan gaan vragen

  # Laat hem een aanvraag doen voor een podiumplekje


env = simpy.Environment()
env.process(set_up(env, pippelaars, colors))
env.run(until=sim_time)

# Condition events
Naast zingen houden onze Pippelaars in de karaokebar ook wel van een biertje. Om de zoveel tijd komt er dan ook een biertje richting de pippelaars. Mocht een pippelaar nou een biertje vasthebben terwijl het zijn/haar beurt wordt om te zingen dan loopt de Pippelaar helaas zijn/haar beurt mis, omdat er alleen nog maar gefocussed werd op het biertje.

**De opdracht:** 

*   Iedere 15 minuten krijgen de Pippelaars een biertje en ze doen er allemaal tussen de 5 en 10 minuten over om dit biertje op te drinken. (Random verdeling tussen de 5 en 10 minuten.) Als een Pippelaar op het podium staat en dan een biertje krijgt is er niks aan de hand en zingt hij/zij lekker door. Maar staat een Pippelaar een biertje te drinken op het moment dat hij/zij aan de beurt is om naar het podium te gaan dan loopt hij/zij deze kans mis.
*   Pippelaars gaan nu niet weg uit de bar nadat ze een nummer hebben gezongen maar blijven gewoon rondhangen. Na 5 tot 35 minuten gaan ze weer een nieuw nummer aanvragen.
* De logica voor een Pippelaar wordt dus als volgt:
> 1. Na binnenkomst ongeveer 10 minuten wachten tot het aanvragen van een nummer
> 2. Nummer aanvragen
> 3. Wachten tot er plaats is op het podium
> 4. Zodra er plaats is op het podium gaan zingen (duurt 10 minuten), tenzij de Pippelaar bier vast heeft, dan vervalt het zingen en gaat de Pippelaar na 5 tot 35 minuten een nieuw nummer aanvragen. (Let op, ook als de Pippelaar wel heeft mogen zingen gaat hij/zij na 5 tot 35 minuten een nieuw nummer aanvragen.)








In [0]:
import random
import simpy
from termcolor import colored


random.seed(42)
pippelaars = ['Ruud', 'Bart', 'Wouter', 'Roos', 'Erlijn', 'Gijs', 'Lars']
sim_time = 150  # Simulate until
colors = ['grey', 'red', 'green', 'blue', 'yellow', 'magenta', 'cyan'] # Kleuren worden toegevoegd zodat output in kleur geprint kan worden voor de overzichtelijkheid


def pippelaar(env, name, podium, color, has_beer):
  """A pippelaar arrives at the karaokebar at a certain time and will request a
  podium space several minutes after his arrival """
  
  yield env.timeout(30)
  # Het enige wat de Pippelaar nu doet is 30 minuten chillen in de karaokebar, dan gaat hij weer weg
  print(colored("%s leaves the bar at %s" % (name, env.now), color))
  
  
def set_up(env, pippelaars, colors, has_beer):
  """ Makes sure Pippelaars arrive at the karaoke bar"""
  podium = simpy.PriorityResource(env, capacity=1)
  env.process(jeroen_arriveert(env, 'Jeroen', podium))
    
  for i in range(3):
    env.process(pippelaar(env, pippelaars[i], podium, colors[i], has_beer))
    print(colored("%s arrived at the karaokebar at %s" % (pippelaars[i], env.now), colors[i]))
    
  for i in range(3, 7):
    yield env.timeout(random.normalvariate(15, 3))
    env.process(pippelaar(env, pippelaars[i], podium, colors[i], has_beer))
    print(colored("%s arrived at the karaokebar at %s" % (pippelaars[i], env.now), colors[i]))
    
  # Zodra alle Pippelaars binnen zijn begint het bier te komen
  env.process(get_beer(env, pippelaars, has_beer, colors))
    
    
def jeroen_arriveert(env, name, podium):
  """Het oprichtertje verdient een apart simpy process om zijn gedrag te simuleren, omdat het anders is dan dat van de andere pippelaars"""
  yield env.timeout(1600) # Nu blijft Jeroen eigenlijk heel de avond door het raam naar binnen kijken zonder de bar in te komen
   
  # Gebruik hier de code van de vorige opdracht
  
    
def get_beer(env,pippelaars, has_beer, colors):
  """ Iedere 15 minuten komt er bier voor iedereen. Iedereen doet er tussen 5 en
  10 minuten over om het op te drinken. """
  
  while True:
    # Nu doet dit niks afgezien van dat dit process steeds een timeout van 15 minuten krijgt
    # Hint: drink_beer moet hier gebruikt worden
    yield env.timeout(15)
    
  
def drink_beer(env, name, has_beer, color):
  has_beer[name].succeed() # Triggered het 'has_beer' event van een Pippelaar
  yield env.timeout(random.randint(5,10))
  print(colored("%s finished his/her drink at %s" %(name, env.now),color))
  has_beer[name] = env.event() # Zet een nieuw 'has_beer' event op de naam van de Pippelaar 
  

has_beer = {pippelaar: env.event() for pippelaar in pippelaars} # Geeft iedereen een event 'has_beer' dat nog niet getriggered is, 
# als dit event wel getriggered is mag de Pippelaar niet het podium op
env = simpy.Environment()
env.process(set_up(env, pippelaars, colors, has_beer))
env.run(until=sim_time)